In [1]:
import sqlite3
import contextlib
import pandas as pd
import numpy as np

In [3]:
sql_connect = sqlite3.connect('database.sqlite')
cursor = sql_connect.cursor()

In [4]:
query_r = "SELECT * FROM sqlite_sequence" # read query
query_i = "INSERT INTO country VALUES ('7819', 'mooi land')" # insert query
query_d = "DELETE FROM country WHERE name='mooi land'" # delete query

In [5]:
sql_connect = sqlite3.connect('database.sqlite')
cursor = sql_connect.cursor()
df = pd.read_sql_query(query_r,sql_connect)
df

,name,seq
0,Team,103916
1,Country,51958
2,League,51958
3,Match,51958
4,Player,11075
5,Player_Attributes,183978
6,Team_Attributes,1458
